# NLP

### Bucket checks

In [27]:
!aws s3 ls


2023-08-29 23:43:16 sagemaker-studio-692960231031-wo7kgoszj2g
2023-08-29 23:50:01 sagemaker-us-east-1-692960231031
2023-08-30 00:34:21 vad49
2023-09-16 16:02:10 vad49-labdata


In [28]:
!aws s3 ls s3://project17-bucket-alex/stories-and-books-nlp/all-model-text/


In [29]:
# save books into bucket if needed
if True is False:
    !aws s3 cp ../../data/external-data/books/ s3://project17-bucket-alex/books --recursive --exclude "*" --exclude ".ipynb_checkpoints/*" --include "*.txt"


### Setup

In [30]:
if True is True: # set to true only for the first un
    # Setup - Run only once per Kernel App
    %conda install openjdk -y

    # install PySpark
    #%pip install pyspark==3.2.0 s3fs pyarrow spark-nlp

    %pip install s3fs pyarrow

        # install PySpark
    %pip install pyspark==3.4.0

    # install spark-nlp
    %pip install spark-nlp==5.1.3

    # restart kernel
    from IPython.core.display import HTML
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached pyspark-3.4.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.2.0
    Uninstalling pyspark-3.2.0:
      Successfully u

In [31]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import col, length, isnan, when, count, regexp_extract, weekofyear, hour, avg, to_date, unix_timestamp, lit, corr

import json
import sparknlp
import numpy as np
import pandas as pd
from sparknlp.base import *
from pyspark.ml import Pipeline
from sparknlp.annotator import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from sparknlp.pretrained import PretrainedPipeline


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 150) 
#pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)


In [32]:
# Import pyspark and build Spark session


spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config("fs.s3a.aws.credentials.provider", "com.amazonaws.auth.ContainerCredentialsProvider")\
    .getOrCreate()

print(spark.version)


3.4.0


### Bring in submissions and comments data

In [33]:
%%time


required_columns = ['subreddit', 'title', 'selftext', 'score', 'created_utc', 'url']


# read the full year

# Read in data from project bucket
bucket = "project17-bucket-alex"

# List of 12 directories each containing 1 month of data
directories = ["project_2022_"+str(i)+"/submissions" for i in range(1,13)]

# Iterate through 12 directories and merge each monthly data set to create one big data set
submissions = None
for directory in directories:
    s3_path = f"s3a://{bucket}/{directory}"
    month_df = spark.read.parquet(s3_path).select(*required_columns)
    
    if submissions is None:
        submissions = month_df
    else:
        submissions = submissions.union(month_df)

        


CPU times: user 49 ms, sys: 1.97 ms, total: 51 ms
Wall time: 5.21 s


In [34]:
submissions_small = submissions.sample(withReplacement=False, fraction=0.001, seed=42)



In [35]:
# create small dfs

use_small = True  # to easily swap between the small and small dfs
submissions_active = submissions_small if use_small else submissions



In [36]:
#cache - only for when working with the small version
#submissions_active.cache()


### Process Submissions Data


In [37]:
submissions_active.printSchema()

root
 |-- subreddit: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- score: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- url: string (nullable = true)



### 2.2 Conduct basic data quality checks! Make sure there are no missing values, check the length of the comments, and remove rows of data that might be corrupted. Even if you think all your data is perfect, you still need to demonstrate that with your analysis.



Let's remove submissions without a body should obviously go, but what about the submissions without a self text (deleted, removed or empty). We can keep where the author is empty.

In [38]:

def clean_submissions(df: DataFrame) -> DataFrame:
    
    # define conditions
    conditions = (col('selftext') != "[removed]") & (col('selftext') != "[deleted]") & (col('selftext').isNotNull() & (col('selftext') != ""))

    
    # apply filter
    cleaned_df = df.filter(conditions)
  

    return cleaned_df




In [39]:
submissions_active = clean_submissions(submissions_active)


In [40]:
#display(f"submissions shape: ({submissions_active.count()}, {len(submissions_active.columns)})")

display(submissions_active.limit(5).toPandas())


,subreddit,title,selftext,score,created_utc,url
0,relationship_advice,I need advice,So my gf and I are both 19 and live separately but both in college and she works. We will live together within 2ish years after she graduates but ...,1,2022-01-25 17:01:32,https://www.reddit.com/r/relationship_advice/comments/scid7f/i_need_advice/
1,relationship_advice,Too smart to be this stupid; logics vs heart.,"My estranged bf of the last almost 2yrs has vowed to make his life mission, along others aid, to make me have misery and regret. \n Call me cr...",1,2022-01-18 17:12:58,https://www.reddit.com/r/relationship_advice/comments/s72fh8/too_smart_to_be_this_stupid_logics_vs_heart/
2,antiwork,High hopes for the future,"Just kidding, this sub is doomed\n\nHey mods, stop doing interviews \n\nYou don’t speak for the people of this sub. You are a janitor who’s only p...",0,2022-01-27 19:05:44,https://www.reddit.com/r/antiwork/comments/se5xf0/high_hopes_for_the_future/
3,NoStupidQuestions,I think I just ejaculated without trying,Okay so has anyone else have this happen. Right altering peeing. I get extreme pain I mean like eye shutting crouching over pain and there this cl...,1,2022-01-05 19:26:45,https://www.reddit.com/r/NoStupidQuestions/comments/rwuuej/i_think_i_just_ejaculated_without_trying/
4,socialskills,"Is it weird to tell my depressed friend I'm ""proud of him"" after he's overcome a bad mental phase?","One of my best friends suffers from depression and experiences ""down phases"" in irregular intervals. He always has a particularly bad one in winte...",131,2022-01-13 18:13:09,https://www.reddit.com/r/socialskills/comments/s35q0j/is_it_weird_to_tell_my_depressed_friend_im_proud/


In [41]:
# use regex the remove text after 'Edit: ' or 'edit: '

# The regular expression pattern
pattern = r"(?i)^(.*?)(?=Edit:|$)"

# Apply the regular expression to create a new column with the modified text
submissions_active = submissions_active.withColumn("selftext_modified", regexp_extract(col("selftext"), pattern, 1))



In [42]:
# define stories as posts longer than a certain length 

story_length = 4500

submissions_active = submissions_active.filter(length(col("selftext")) > story_length)


In [43]:
# keep only the 25% most engaging posts


# Calculate the approximate percentile of the 'score' column
quantile_value = submissions_active.approxQuantile("score", [0.85], 0.05)  # 0.05 is the relative error

# Filter the DataFrame to keep scores above or equal to this value
submissions_active = submissions_active.filter(col("score") >= quantile_value[0])



In [44]:
#display(f"submissions shape: ({submissions_active.count()}, {len(submissions_active.columns)})")


display(submissions_active.limit(5).toPandas())

,subreddit,title,selftext,score,created_utc,url,selftext_modified
0,relationship_advice,Is dating online cheating.. the whole story,Due to the overwhelming messages and requests for the whole story on about my last post. I decided to do a tell-all. So I hope you have a minute. ...,29,2022-07-13 15:48:56,https://www.reddit.com/r/relationship_advice/comments/vy6fh0/is_dating_online_cheating_the_whole_story/,
1,AmItheAsshole,AITA for expecting my son to share his room?,"Background: My (40sf) husband (40sm) and I bought a 3-bedroom house a few years ago, shortly before the panini. We of course took the master bedro...",16459,2022-10-26 08:51:01,https://www.reddit.com/r/AmItheAsshole/comments/ydt2w5/aita_for_expecting_my_son_to_share_his_room/,
2,AmItheAsshole,"WIBTA for taking my (15f) laundry basket full of clothes, which my stepdad (49m) threw out for not putting them away, out of the trash?",Sorry for bad English!\n\nThe morning before my clothes were thrown away my mom put them in a laundry basket beside my door. She did tell me ofcou...,95,2022-11-07 19:59:01,https://www.reddit.com/r/AmItheAsshole/comments/yoyf1r/wibta_for_taking_my_15f_laundry_basket_full_of/,
3,relationship_advice,My housemate (25f) did something I (20f) consider morally unacceptable and its causing a lot of conflict in the house (6 students).,Tl;dr version: I live in a student houseshare. One of my housemates is a student nurse and was extremely inappropriate with a patient which I foun...,421,2022-11-04 20:32:06,https://www.reddit.com/r/relationship_advice/comments/ym9kw7/my_housemate_25f_did_something_i_20f_consider/,
4,socialskills,How to Talk About Yourself (and How to Have Good Conversation),"**TL:DR talk about yourself by saying just a little bit in a way that's relevant to the topic at hand, and give the other person the implicit choi...",27,2022-12-01 14:46:48,https://www.reddit.com/r/socialskills/comments/z9polj/how_to_talk_about_yourself_and_how_to_have_good/,


Books

In [45]:
import os
from pyspark.sql import Row

def process_gutenberg_books_to_df(directory_path):
    all_rows = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Assuming the files are in .txt format
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                start_idx = content.find("*** START OF THE PROJECT GUTENBERG")
                if start_idx != -1:
                    start_idx = content.find('\n', start_idx) + 1
                    content = content[start_idx:]
                # Splitting the content into paragraphs or sentences
                for paragraph in content.split('\n\n'):
                    if paragraph.strip():  # Check if the paragraph is not just whitespace
                        all_rows.append(Row(text=paragraph.strip()))

    # Create a DataFrame from the list of Rows
    return spark.createDataFrame(all_rows)

# Process the Gutenberg books and create a DataFrame
books = process_gutenberg_books_to_df('../../data/external-data/books')
books.show()


+--------------------+
|                text|
+--------------------+
|       Metamorphosis|
|      by Franz Kafka|
|Translated by Dav...|
|                   I|
|One morning, when...|
|“What’s happened ...|
|Gregor then turne...|
|“Oh, God”, he tho...|
|He slid back into...|
|And he looked ove...|
|He was still hurr...|
|The first thing h...|
|It was a simple m...|
|The first thing h...|
|So then he tried ...|
|It took just as m...|
|But then he said ...|
|When Gregor was a...|
|After a while he ...|
|“Something’s fall...|
+--------------------+
only showing top 20 rows



23/11/18 15:25:53 WARN TaskSetManager: Stage 39 contains a task of very large size (1439 KiB). The maximum recommended task size is 1000 KiB.


In [46]:
from pyspark.sql.functions import concat_ws, col

# Process `submissions_active` to extract and transform the 'selftext' column
submissions_active = submissions_active.select("selftext")





In [47]:
# For the Project Gutenberg DataFrame
#books = books.withColumnRenamed("value", "text")

# For the submissions DataFrame
submissions_active = submissions_active.withColumnRenamed("selftext", "text")

# Now you can union them
all_model_text = books.unionByName(submissions_active)

In [48]:
all_model_text.show()

+--------------------+
|                text|
+--------------------+
|       Metamorphosis|
|      by Franz Kafka|
|Translated by Dav...|
|                   I|
|One morning, when...|
|“What’s happened ...|
|Gregor then turne...|
|“Oh, God”, he tho...|
|He slid back into...|
|And he looked ove...|
|He was still hurr...|
|The first thing h...|
|It was a simple m...|
|The first thing h...|
|So then he tried ...|
|It took just as m...|
|But then he said ...|
|When Gregor was a...|
|After a while he ...|
|“Something’s fall...|
+--------------------+
only showing top 20 rows



23/11/18 15:25:54 WARN TaskSetManager: Stage 40 contains a task of very large size (1439 KiB). The maximum recommended task size is 1000 KiB.


Clean

In [49]:

#########
#clean
#########


from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType
import string

# Start Spark NLP pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setLowercase(True) \

finisher = Finisher() \
    .setInputCols(["normalized"]) \
    .setOutputCols(["clean_text"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(True)

# Define the pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline().setStages([
    document_assembler,
    tokenizer,
    normalizer,
    finisher
])

# Apply the pipeline to the submissions and books DataFrames
#submissions_active_df = pipeline.fit(submissions_active).transform(submissions_active)
all_model_text = pipeline.fit(all_model_text).transform(all_model_text)


def chars_to_ints(text):
    vocab = sorted(set("".join(text)))  # Create a vocabulary from the cleaned text
    char2idx = {c: i for i, c in enumerate(vocab)}
    return [char2idx.get(c, 0) for c in "".join(text)]  # Convert each char in text to its integer representation

chars_to_ints_udf = udf(chars_to_ints, ArrayType(IntegerType()))

# Apply the UDF to the 'clean_text' column
#submissions_active_df = submissions_active_df.withColumn("text_as_int", chars_to_ints_udf(col("clean_text")))
all_model_text = all_model_text.withColumn("text_as_int", chars_to_ints_udf(col("clean_text")))



In [50]:
all_model_text.show(25)

23/11/18 15:25:55 WARN TaskSetManager: Stage 41 contains a task of very large size (1439 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+
|                text|          clean_text|         text_as_int|
+--------------------+--------------------+--------------------+
|       Metamorphosis|     [metamorphosis]|[4, 1, 9, 0, 4, 5...|
|      by Franz Kafka|  [by, franz, kafka]|[1, 6, 2, 5, 0, 4...|
|Translated by Dav...|[translated, by, ...|[9, 7, 0, 6, 8, 5...|
|                   I|                 [i]|                 [0]|
|One morning, when...|[one, morning, wh...|[13, 12, 4, 11, 1...|
|“What’s happened ...|[whats, happened,...|[19, 7, 0, 16, 15...|
|Gregor then turne...|[gregor, then, tu...|[6, 16, 4, 6, 13,...|
|“Oh, God”, he tho...|[oh, god, he, tho...|[13, 7, 6, 13, 3,...|
|He slid back into...|[he, slid, back, ...|[7, 4, 17, 11, 8,...|
|And he looked ove...|[and, he, looked,...|[0, 12, 3, 7, 4, ...|
|He was still hurr...|[he, was, still, ...|[7, 4, 21, 0, 17,...|
|The first thing h...|[the, first, thin...|[17, 7, 4, 5, 8, ...|
|It was a simple m...|[it

In [51]:
all_model_text.write.mode("overwrite").format("parquet").save("s3a://project17-bucket-alex/stories-and-books-nlp/")





23/11/18 15:25:57 WARN TaskSetManager: Stage 42 contains a task of very large size (1439 KiB). The maximum recommended task size is 1000 KiB.
